In [2]:
!pip install music21 pretty_midi 
#tensorflow keras midi2audio
# !apt-get install -y fluidsynth

     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
     - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
     --- ------------------------------------ 0.5/5.6 MB 509.0 kB/s eta 0:00:10
     --- ------------------------------------ 0.5/5.6 MB 509.0 kB/s eta 0:00:10
     --- ------------------------------------ 0.5/5.6 MB 509.0 kB/s eta 0:00:10
     --- ------------------------------------ 0.5/5.6 MB 509.0 kB/s eta 0:00:10
     ----- ---------------------------------- 0.8/5.6 MB 409.3 kB/s eta 0:00:12
     ----- ---------------------------------- 0.8/5.6 MB 409.3 kB/s eta 0:00:12
     ----- ---------------------------------- 0.8/5.6 MB 409.3 kB/s eta 0:00:12
     ----- ---

  DEPRECATION: Building 'pretty_midi' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pretty_midi'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
PROJECT_DIR = '/content/drive/MyDrive/MusicAI/'
os.makedirs(PROJECT_DIR, exist_ok=True)
print('Project directory:', PROJECT_DIR)

Project directory: /content/drive/MyDrive/MusicAI/


In [1]:
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
# from midi2audio import FluidSynth
from IPython.display import Audio
from tensorflow.keras.utils import plot_model
from keras.models import load_model

In [ ]:
MIDI_GLOB = PROJECT_DIR + "*.mid"
print('Searching for MIDI files at:', MIDI_GLOB)


midi_files = glob.glob(MIDI_GLOB)
if len(midi_files) == 0:
  print('No MIDI files found. Please upload .mid files to the folder above and re-run this cell.')
else:
  print(f'Found {len(midi_files)} MIDI files. Parsing...')

from tqdm.notebook import tqdm

notes = []
for file in tqdm(midi_files):
  try:
    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)
    notes_to_parse = parts.parts[0].recurse() if parts else midi.flat.notes


    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
  except Exception as e:
    print('Warning: failed parsing', file, ' — ', e)


print(f'Total notes parsed: {len(notes)}')


Searching for MIDI files at: /content/drive/MyDrive/MusicAI/*.mid
Found 385 MIDI files. Parsing...


  0%|          | 0/385 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2004 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2002 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEve

In [ ]:
import tensorflow as tf

gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available.")
else:
    print("No GPU available. The code will run on the CPU.")

In [ ]:
sequence_length = 100
if len(notes) < sequence_length:
  raise ValueError(f'Not enough notes ({len(notes)}) to create sequences of length {sequence_length}. Use more MIDI files or reduce sequence_length.')


pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)
print('Unique tokens (vocab size):', n_vocab)
import pickle
with open(PROJECT_DIR + "notes_vocab.pkl", "wb") as f:
    pickle.dump(pitchnames, f)



note_to_int = dict((note, number) for number, note in enumerate(pitchnames))


network_input = []
network_output = []
for i in range(len(notes) - sequence_length):
  seq_in = notes[i:i + sequence_length]
  seq_out = notes[i + sequence_length]
  network_input.append([note_to_int[ch] for ch in seq_in])
  network_output.append(note_to_int[seq_out])


n_patterns = len(network_input)
print('Total training patterns:', n_patterns)


X = np.reshape(network_input, (n_patterns, sequence_length, 1))
X = X / float(n_vocab)
y = to_categorical(network_output)
print('X shape:', X.shape, 'y shape:', y.shape)

In [ ]:
model = Sequential([
  LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
  Dropout(0.3),
  LSTM(512, return_sequences=True),
  Dropout(0.3),
  LSTM(512, return_sequences=True),
  Dropout(0.3),
  LSTM(512),
  BatchNorm(),
  Dense(512),
  Activation('relu'),
  Dropout(0.3),
  Dense(n_vocab),
  Activation('softmax')
])


model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()
model.save(PROJECT_DIR + "full_model.keras")

In [ ]:
import pickle
with open(PROJECT_DIR + "notes_vocab.pkl", "wb") as f:
    pickle.dump(pitchnames, f)
with open(PROJECT_DIR + "seq_length.pkl", "wb") as f:
    pickle.dump(sequence_length, f)

In [ ]:
WEIGHTS_PATH = PROJECT_DIR + 'model_weights.keras'
checkpoint = ModelCheckpoint(WEIGHTS_PATH, monitor='loss', save_best_only=True, mode='min')


# You can adjust epochs & batch_size depending on GPU/time availability
model.fit(X, y, epochs=100, batch_size=64, callbacks=[checkpoint])


print('Training finished. Weights saved to', WEIGHTS_PATH)

#Generation

In [ ]:
WEIGHTS_PATH = PROJECT_DIR + 'model_weights.keras'
if os.path.exists(WEIGHTS_PATH):
    model.load_weights(WEIGHTS_PATH)
    print('Loaded weights from', WEIGHTS_PATH)
else:
    print('Weights file not found. You can still try to generate from the current model (untrained).')


start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start].copy()
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))


prediction_output = []
num_generate = 300 # change to longer/shorter as desired
for note_index in range(num_generate):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)


    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)


    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
output_midi_path = PROJECT_DIR + 'AI_Music_Output.mid'
offset = 0
output_notes = []
for pattern in prediction_output:
  if ('.' in pattern) or pattern.isdigit():
    notes_in_chord = pattern.split('.')
    notes_objs = [note.Note(int(n)) for n in notes_in_chord]
    for n in notes_objs:
      n.storedInstrument = instrument.Piano()
    new_chord = chord.Chord(notes_objs)
    new_chord.offset = offset
    output_notes.append(new_chord)
  else:
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)
  offset += 0.5


midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp=output_midi_path)
print('Saved generated MIDI to', output_midi_path)

In [ ]:
output_wav_path = PROJECT_DIR + 'AI_Music_Output.wav'
fs = FluidSynth()
fs.midi_to_audio(output_midi_path, output_wav_path)
print('Saved WAV to', output_wav_path)


# In Colab this will embed an audio player
Audio(output_wav_path)